In [ ]:
# GET PARAMETERS
import os 
import sys
# Get Parent folder : 
current_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_path, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from examples.train_and_visu_non_recurrent import evaluate_config,get_ds,train_the_config,get_ds_without_shuffling_on_train_set
from plotting.plotting import error_per_station_calendar_pattern
from examples.train_model_on_k_fold_validation import load_configuration

In [ ]:
#'subway_in_STGCN_MSELoss_2025_01_06_08_00_94523',
#"subway_in_netmob_POIs_STGCN_VariableSelectionNetwork_MSELoss_2025_01_09_07_54_72902" # contient  Instagram, Google Maps, Deezer, WhatsApps, Twiteter, DL, UL
trial_id ='subway_in_STGCN_MSELoss_2025_01_20_14_27_20569'
args_init,_ = load_configuration(trial_id,load_config=True,epochs=None)

fold_to_evaluate = [args_init.K_fold-1]
modification = {}


trainer,ds,args,trial_id,df_loss = train_the_config(modification,args_init,fold_to_evaluate)
trainer,ds_no_shuffle = get_ds_without_shuffling_on_train_set(trainer,modification,args_init, fold_to_evaluate)

station = ['BEL','PER','PAR','GER','CHA']   # 'BON'  #'GER'
training_mode_to_visualise = ['test']#,'valid','train']
modification ={'keep_best_weights':True,
                'epochs':100,
                'validation_split_method' : 'forward_chaining_cv',
                'min_fold_size_proportion': 0.75,
                'train_prop':0.6,
                'valid_prop':0.2,
                'test_prop':0.2,

                'set_spatial_units':station,
        
                'data_augmentation': True, #True,  #False
                'DA_method':'interpolation', # 'noise' # 'interpolation
                'DA_moment_to_focus' : None, #[{'hours':[0,23],'weekdays':[1,3]}], # None
                }
if False:
    ds,args_modif,trial_id,save_folder,df_loss = get_ds(args.model_name,args.dataset_names,args.dataset_for_coverage,
                                                modification=modification,args_init=args,fold_to_evaluate=[args.K_fold-1])

(trainer,ds,ds_no_shuffle,args) = evaluate_config(args_best_model.model_name,args_best_model.dataset_names,args_best_model.dataset_for_coverage,
                                                station = station,
                                                modification=modification,
                                                training_mode_to_visualise=training_mode_to_visualise,
                                                args_init =args_best_model,
                                                fold_to_evaluate =[args_best_model.K_fold-1])

# Init
for training_mode in training_mode_to_visualise:
    min_flow = 20  # Minimal Flow considered for MAPE, otherwise set error = 0
    limit_percentage_error = 200 # 300% plus mauvais que quand on se sert du previous 
    fig,axes = error_per_station_calendar_pattern(trainer,ds,training_mode,metrics = ['mse','mae','mape','previous_value'],
                                                freq='1h',
                                                min_flow=min_flow,
                                                figsize = (30,5*len(station)),
                                                limit_percentage_error = limit_percentage_error,
                                                stations = station)